In [14]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
from collections import OrderedDict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
df = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/Finance/master/dataset/Walmart%20Stock%20Price%20from%201972-2022/WMT.csv").set_index("Date")
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1972-08-25,0.063477,0.064697,0.063477,0.064453,0.036447,2508800
1972-08-28,0.064453,0.064941,0.064209,0.064209,0.036309,972800
1972-08-29,0.063965,0.063965,0.063477,0.063477,0.035895,1945600
1972-08-30,0.063477,0.063477,0.062988,0.063477,0.035895,409600
1972-08-31,0.062988,0.062988,0.062500,0.062500,0.035343,870400


In [17]:
class WLMRTSSTOCKPRICE(Dataset):
    def __init__(self, split = "Train"):
        super(WLMRTSSTOCKPRICE, self).__init__()
        df = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/Finance/master/dataset/Walmart%20Stock%20Price%20from%201972-2022/WMT.csv").set_index("Date").sample(frac = 1)
        self.df_x = df[["Open", "High", "Low"]]
        self.df_y = df[["Close", "Adj Close"]]
        
        if split == "Train":
            self.x = torch.from_numpy(self.df_x.iloc[:10000].to_numpy()).type(torch.FloatTensor)
            self.y = torch.from_numpy(self.df_y.iloc[:10000].to_numpy()).type(torch.FloatTensor)
            self.len = self.x.shape[0]
        elif split == "Valid":
            self.x = torch.from_numpy(self.df_x.iloc[10000:12000].to_numpy()).type(torch.FloatTensor)
            self.y = torch.from_numpy(self.df_y.iloc[10000:12000].to_numpy()).type(torch.FloatTensor)
            self.len = self.x.shape[0]
        elif split == "Test":
            self.x = torch.from_numpy(self.df_x.iloc[12000:].to_numpy()).type(torch.FloatTensor)
            self.y = torch.from_numpy(self.df_y.iloc[12000:].to_numpy()).type(torch.FloatTensor)
            self.len = self.x.shape[0]
        else:
            print("Unknown Split")
        
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
train_dataset = WLMRTSSTOCKPRICE(split = "Train")
valid_dataset = WLMRTSSTOCKPRICE(split = "Valid")
test_dataset = WLMRTSSTOCKPRICE(split = "Test")

train_loader = DataLoader(train_dataset, batch_size = 128, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False)
class WLMRTNET(nn.Module):
    def __init__(self, input_size, number_of_classes, p = 0.1):
        super(WLMRTNET, self).__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(inplace = True),
            nn.Dropout(p = p)
        )
        
        self.h1 = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(inplace = True),
            nn.Dropout(p = p)
        )
        
        self.h2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(inplace = True),
            nn.Dropout(p = p)
        )
        
        self.h3 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace = True),
            nn.Dropout(p = p)
        )
        
        self.h4 = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(inplace = True),
            nn.Dropout(p = p)
        )
        
        self.output_layer = nn.Sequential(
            nn.Linear(64, number_of_classes)
        )
        
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        x = F.relu(self.h3(x))
        x = F.relu(self.h4(x))
        x = self.output_layer(x)
        return x
model = WLMRTNET(3, 2, 0.0)
LEARNING_RATE = 0.0001
EPOCHS = 100
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = 0.001)
min_loss = np.inf

model.to(device)

WLMRTNET(
  (input_layer): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.0, inplace=False)
  )
  (h1): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.0, inplace=False)
  )
  (h2): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.0, inplace=False)
  )
  (h3): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.0, inplace=False)
  )
  (h4): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.0, inplace=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [18]:
def train(model, epochs, optimizer, criterion, min_loss):
    training_losses, valid_losses, accs = [],[],[]
    for epoch in range(epochs):
        train_loss = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            yhat = model(x)
            loss = criterion(yhat, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        training_losses.append(train_loss)
        
        with torch.no_grad():
            valid_loss = 0
            for x, y in valid_loader:
                x, y = x.to(device), y.to(device)
                yhat = model(x)
                loss = criterion(yhat, y)
                
                valid_loss += loss.item()
                
            valid_losses.append(valid_loss)
                
        print("EPOCH: {}.. \tTRAINING LOSS: {:.6F}.. \tVALID LOSS: {:.6F}".format(epoch + 1, train_loss, valid_loss))
        
        if valid_loss <= min_loss:
            print("Saving Model: {:.4f} ---> {:.4f}".format(min_loss, valid_loss))
            save_obj = OrderedDict([
                ("model", model.state_dict()),
                ("min_loss", min_loss)
            ])
            torch.save(save_obj, "/kaggle/working/wlmrt.pt")
            min_loss = valid_loss
            
    return training_losses, valid_loss, accs

In [19]:
training_loss, valid_loss, accs = train(model, EPOCHS, optimizer, criterion, min_loss)

EPOCH: 1.. 	TRAINING LOSS: 84632.871088.. 	VALID LOSS: 644.720412
Saving Model: inf ---> 644.7204
EPOCH: 2.. 	TRAINING LOSS: 1528.496832.. 	VALID LOSS: 602.878479
Saving Model: 644.7204 ---> 602.8785
EPOCH: 3.. 	TRAINING LOSS: 1433.758623.. 	VALID LOSS: 586.913805
Saving Model: 602.8785 ---> 586.9138
EPOCH: 4.. 	TRAINING LOSS: 1374.531817.. 	VALID LOSS: 560.584536
Saving Model: 586.9138 ---> 560.5845
EPOCH: 5.. 	TRAINING LOSS: 1317.468917.. 	VALID LOSS: 505.289371
Saving Model: 560.5845 ---> 505.2894
EPOCH: 6.. 	TRAINING LOSS: 1150.766778.. 	VALID LOSS: 528.198351
EPOCH: 7.. 	TRAINING LOSS: 1009.820642.. 	VALID LOSS: 354.348148
Saving Model: 505.2894 ---> 354.3481
EPOCH: 8.. 	TRAINING LOSS: 725.327001.. 	VALID LOSS: 385.487185
EPOCH: 9.. 	TRAINING LOSS: 612.037677.. 	VALID LOSS: 174.982761
Saving Model: 354.3481 ---> 174.9828
EPOCH: 10.. 	TRAINING LOSS: 355.973829.. 	VALID LOSS: 124.956701
Saving Model: 174.9828 ---> 124.9567
EPOCH: 11.. 	TRAINING LOSS: 266.442605.. 	VALID LOSS: 257.74

In [20]:
count = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        yhat = model(x)
        
        if count % 100 == 0:
            print("Predicted Close: {:.3f} True Close: {:.3f} \tPredicted Adj Close: {:.3f} True Adj Close: {:.3f}".format(yhat[0][0].item(), y[0][0].item(), yhat[0][1].item(), y[0][1].item()))
        count += 1

Predicted Close: 0.113 True Close: 0.066 	Predicted Adj Close: -0.008 True Adj Close: 0.038
Predicted Close: 51.521 True Close: 50.900 	Predicted Adj Close: 36.679 True Adj Close: 37.843
Predicted Close: 108.120 True Close: 107.520 	Predicted Adj Close: 103.360 True Adj Close: 103.127
Predicted Close: 0.095 True Close: 0.024 	Predicted Adj Close: -0.018 True Adj Close: 0.014
Predicted Close: 0.100 True Close: 0.042 	Predicted Adj Close: -0.015 True Adj Close: 0.024
